In [1]:
import json
import keras_tuner as kt
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

---
### Data Reading
---

In [2]:
data = np.load('../data/training_data_arg.npz', allow_pickle=True)

train_set = data['train_set']
test_set = data['test_set']
val_set = data['val_set']

In [3]:
def create_dataframe(set):
    df_ = pd.DataFrame([], columns=['x', 'y'])	
    for i in range(len(set)):
        df_ = pd.concat([df_, pd.DataFrame(
            {'reading': [set[i]['x'].reshape(24, 32)],
            'posture': set[i]['y']}
        )])
    return df_

In [4]:
df_train = create_dataframe(train_set)
df_test = create_dataframe(test_set)
df_val = create_dataframe(val_set)

print(df_train.shape, df_test.shape, df_val.shape)

(1620, 4) (54, 4) (405, 4)


In [5]:
X_train = df_train['reading']
y_train = df_train['posture']
X_val = df_val['reading']
y_val = df_val['posture']
X_test = df_test['reading']
y_test = df_test['posture']
# One-hot encode the labels
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_val).values
y_test = pd.get_dummies(y_test).values

In [6]:
X_train

0    [[24.77, 25.37, 25.45, 26.17, 26.53, 29.51, 30...
0    [[25.74, 26.52, 27.07, 28.7, 30.48, 32.82, 32....
0    [[26.5, 27.57, 28.01, 29.04, 30.02, 33.2, 33.1...
0    [[25.21, 25.82, 25.93, 26.43, 27.04, 28.96, 29...
0    [[25.85, 26.64, 27.07, 28.09, 28.72, 31.57, 32...
                           ...                        
0    [[24.078905745999865, 23.54478085637246, 24.05...
0    [[24.287305549063102, 24.46721305051553, 24.36...
0    [[23.187623444806476, 23.419400611664578, 23.4...
0    [[24.389370902565755, 24.016248933303498, 24.0...
0    [[24.309337096494875, 24.68560284513844, 24.62...
Name: reading, Length: 1620, dtype: object

In [7]:
X_train = np.array([np.array(x) for x in X_train])
X_train = np.expand_dims(X_train, -1)
X_val = np.array([np.array(x) for x in X_val])
X_val = np.expand_dims(X_val, -1)
X_test = np.array([np.array(x) for x in X_test])
X_test = np.expand_dims(X_test, -1)
y_train = np.array([np.array(x) for x in y_train])
y_test = np.array([np.array(x) for x in y_test])
y_val = np.array([np.array(x) for x in y_val])
# Convert the data to tensors
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

---
### Model Definition
---

In [8]:
def model_builder(hp):
    hp_channels_1 = hp.Int('channels_1', min_value=4, max_value=16, step=4)
    hp_channels_2 = hp.Int('channels_2', min_value=4, max_value=8, step=4)
    hp_dense_1 = hp.Int('dense_1', min_value=16, max_value=64, step=16)
    hp_dense_2 = hp.Int('dense_2', min_value=16, max_value=32, step=16)
    model = keras.Sequential(
        [
            layers.Conv2D(hp_channels_1, kernel_size=(3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(hp_channels_2, kernel_size=(3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),  # Flatten the output of the CNN
            layers.Dense(hp_dense_1, activation='relu'),
            layers.Dense(hp_dense_2, activation='relu'),
            layers.Dense(3, activation='softmax'),
        ]
    )
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'],
    )
    return model

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    patience=5
)

In [10]:
tuner = kt.GridSearch(
    model_builder,
    objective='val_loss',
    max_retries_per_trial=5,
    overwrite=True,
    directory='gridsearch-laguardia-DA',
    project_name='posture',
)

---
### Model Training
---

In [11]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr], epochs=500, batch_size=32)

Trial 64 Complete [00h 00m 11s]
val_accuracy: 0.8345679044723511

Best val_accuracy So Far: 0.8864197731018066
Total elapsed time: 00h 11m 17s


In [12]:
tuner.results_summary()

Results summary
Results in gridsearch-laguardia-DA/posture
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0023 summary
Hyperparameters:
channels_1: 8
channels_2: 4
dense_1: 64
dense_2: 32
Score: 0.8864197731018066

Trial 0054 summary
Hyperparameters:
channels_1: 16
channels_2: 4
dense_1: 64
dense_2: 16
Score: 0.8864197731018066

Trial 0046 summary
Hyperparameters:
channels_1: 12
channels_2: 8
dense_1: 64
dense_2: 16
Score: 0.8814814686775208

Trial 0058 summary
Hyperparameters:
channels_1: 16
channels_2: 8
dense_1: 32
dense_2: 16
Score: 0.8716049194335938

Trial 0031 summary
Hyperparameters:
channels_1: 8
channels_2: 8
dense_1: 64
dense_2: 32
Score: 0.8666666746139526

Trial 0006 summary
Hyperparameters:
channels_1: 4
channels_2: 4
dense_1: 64
dense_2: 16
Score: 0.8617283701896667

Trial 0009 summary
Hyperparameters:
channels_1: 4
channels_2: 8
dense_1: 16
dense_2: 32
Score: 0.8617283701896667

Trial 0028 summary
Hyperparameters:
channels_1: 8
channels_2:

---
### Model Evaluation
---

In [13]:
tf.get_logger().setLevel('ERROR')
best_models = tuner.get_best_models(num_models=10)

for i, model in enumerate(best_models):
    print(i, model.evaluate(X_test, y_test))

2/2 [==============================] - 0s 3ms/step - loss: 0.6601 - accuracy: 0.8889
0 [0.660110592842102, 0.8888888955116272]
2/2 [==============================] - 0s 2ms/step - loss: 0.4931 - accuracy: 0.9074
1 [0.49310797452926636, 0.9074074029922485]
2/2 [==============================] - 0s 2ms/step - loss: 0.4898 - accuracy: 0.8704
2 [0.4897642135620117, 0.8703703880310059]
2/2 [==============================] - 0s 2ms/step - loss: 0.6566 - accuracy: 0.8333
3 [0.6566040515899658, 0.8333333134651184]
2/2 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.8333
4 [0.6899811029434204, 0.8333333134651184]
2/2 [==============================] - 0s 2ms/step - loss: 0.6705 - accuracy: 0.8519
5 [0.6705048680305481, 0.8518518805503845]
2/2 [==============================] - 0s 2ms/step - loss: 0.3956 - accuracy: 0.9259
6 [0.39558833837509155, 0.9259259104728699]
2/2 [==============================] - 0s 2ms/step - loss: 0.9501 - accuracy: 0.7963
7 [0.9501034617424

In [16]:
best_model = tuner.get_best_models(10)[6]
best_model.save('best_cmodel_DA.keras')

In [17]:
best_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3956 - accuracy: 0.9259


[0.39558833837509155, 0.9259259104728699]